## Purpose of our Program: 
The purpose of our program is for readers to get reccomendations, provide reviews, and find new books to read.

### Purpose of my feature:
The purpose of my feature, the wishlist, is for readers to be able to keep track of the books that they want to read in the future.

### Lists
Lists are used to handle rows of data, such as when restoring the Wishlist table from a list of dictionaries. Here, data is a list of dictionaries, each representing a row to be added to the database.
The restore method iterates over the list and adds each dictionary as a row in the database.

In [ ]:
data = [{"book_id": 1}, {"book_id": 2}, {"book_id": 3}]
Wishlist.restore(data)

### Dictionaries
Each dictionary contains column names as keys (id, book_id) and their respective values.
This is particularly useful when serializing database objects for use in APIs, such as returning JSON responses.

In [ ]:
def read(self):
    return {
        "id": self.id,
        "book_id": self.book_id,
    }

### CRUD
The function get_wishlist() retrieves all rows from the wishlist table as a list of Wishlist objects. The SQLAlchemy ORM is the third-party library used here.

In [ ]:
def get_wishlist():
    return Wishlist.query.all()

The returned list can be further processed, e.g., converted into a JSON response. This converts the list of Wishlist objects into a list of dictionaries. 


In [ ]:
wishlist_items = [item.read() for item in get_wishlist()]


The Wishlist model provides methods for handling individual columns (id, book_id):

Create: Using add_to_wishlist(book_id):


In [ ]:
item = Wishlist(book_id=book_id)
db.session.add(item)
db.session.commit()

Read: Using read():


In [ ]:
wishlist_dict = item.read()


Delete: Using delete_from_wishlist(book_id):


In [ ]:
item = Wishlist.query.filter_by(book_id=book_id).first()
db.session.delete(item)
db.session.commit()

# Algorithmic Code Request

## Code Blocks to Handle a Request
To handle requests, a Python API class can be created using frameworks like Flask. The class would define methods for handling `GET`, `POST`, `PUT`, and `DELETE` requests, which correspond to the standard HTTP methods used for interacting with resources.

### Example API Class
```python
from flask import Flask, jsonify, request

app = Flask(__name__)

class API:
    def __init__(self):
        self.data = {}

    def get(self, key):
        if key in self.data:
            return jsonify({"key": key, "value": self.data[key]}), 200
        return jsonify({"error": "Key not found"}), 404

    def post(self):
        body = request.json
        key = body.get("key")
        value = body.get("value")
        if key and value:
            self.data[key] = value
            return jsonify({"message": "Created successfully", "data": {key: value}}), 201
        return jsonify({"error": "Invalid input"}), 400

    def put(self, key):
        if key in self.data:
            body = request.json
            value = body.get("value")
            if value:
                self.data[key] = value
                return jsonify({"message": "Updated successfully", "data": {key: value}}), 200
            return jsonify({"error": "Invalid input"}), 400
        return jsonify({"error": "Key not found"}), 404

    def delete(self, key):
        if key in self.data:
            del self.data[key]
            return jsonify({"message": "Deleted successfully"}), 200
        return jsonify({"error": "Key not found"}), 404

api_instance = API()

@app.route('/data/<key>', methods=['GET'])
def get_data(key):
    return api_instance.get(key)

@app.route('/data', methods=['POST'])
def create_data():
    return api_instance.post()

@app.route('/data/<key>', methods=['PUT'])
def update_data(key):
    return api_instance.put(key)

@app.route('/data/<key>', methods=['DELETE'])
def delete_data(key):
    return api_instance.delete(key)

if __name__ == '__main__':
    app.run(debug=True)
```

## Method Containing Sequencing, Selection, and Iteration
The `post` method demonstrates sequencing, selection, and iteration as follows:
- **Sequencing**: The method executes statements in order, such as extracting JSON data, validating input, and updating the dataset.
- **Selection**: Conditional checks (e.g., `if key and value`) ensure that only valid input is processed.
- **Iteration**: While the method itself does not directly iterate, similar logic could iterate over keys in a batch operation.

### Parameters and Return Type
- **Parameters**: The `post` method extracts data from the `request.json` object, which includes a `key` and `value`.
- **Return Type**: Responses are returned using `jsonify`, which ensures a JSON format response.

## Call to Algorithm Request

### Definition of Code Block to Make a Request
A request to the API can be made using tools like Postman or programmatically using Python's `requests` library.

```python
import requests

# Example POST request
response = requests.post('http://localhost:5000/data', json={"key": "example", "value": "value1"})
print(response.json())

# Example GET request
response = requests.get('http://localhost:5000/data/example')
print(response.json())
```

## Call/Request to the Method
When calling the `post` endpoint:
1. The client sends JSON data to the server.
2. The server processes the input, updates the `data` dictionary, and returns a success message or error response.

### Handling Responses
Responses from the API are handled using JSON parsing, allowing client applications to render or act on returned data.

## Changing Data or Method to Trigger Responses
### Normal Conditions
- **POST**: Sending valid key-value pairs updates the dataset and returns a 201 status.
- **GET**: Requesting an existing key retrieves the value with a 200 status.

### Error Conditions
- **POST**: Omitting a `key` or `value` returns a 400 error.
- **GET**: Requesting a non-existent key returns a 404 error.

### Example Response Handling
```python
response = requests.get('http://localhost:5000/data/nonexistent')
if response.status_code == 404:
    print("Key not found")
else:
    print(response.json())
```
By adjusting the input or endpoint, different conditions can be tested to ensure robust error handling and consistent functionality.

